[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io/)

# Image Processing with Python

## Color Spaces

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 06/10/2023 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/ImageProcessingPython/0004ColorSpace.ipynb)

In [1]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

from numba import jit, njit

# Image Processing
import skimage as ski

# Machine Learning


# Miscellaneous
import os
from platform import python_version
import random
import timeit

# Typing
from typing import Callable, List, Tuple

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
# from bokeh.plotting import figure, show

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatRangeSlider, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

### Code Exercise

 - Single line fill

 ```python
 vallToFill = ???
 ```

 - Multi Line to Fill (At least one)

 ```python
 # You need to start writing
 ????
 ```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

???
#===============================================================#
```

In [2]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [3]:
# Constants



In [4]:
# Fixel Algorithms Packages


In [5]:
# General Auxiliary Functions



## Color Space

The _Color Space_ define the representation of the image information.  
Different spaces have different properties for different applications.

 - Image Enhancement: Speed, Luminosity / Chroma Separation.  
   `YUV`, `YCgCr`, `LAB`.
 - Image Masking (Luminosity / Chroma): Hue / Saturation Representation.  
   `HSV` / `HSL`, `LCH`.
 - Vision: Perceptually Consistent Metrics.  
   `LAB`, `LUV`, `OKLab`, `HTCLab`.

This notebooks exercises some color spaces usages.  
It focuses on building "application" for 3 channels masking of an image.

* <font color='brown'>(**#**)</font> There are packages which are far more capable in color transformations: [`Colour`](https://github.com/colour-science/colour), [`Coloria`](https://github.com/coloria-dev/coloria).
* <font color='brown'>(**#**)</font> In applications where run time is crucial the color spaces used are those defined by a linear transformation.
* <font color='brown'>(**#**)</font> In Vision, the color space was crucial in time prior to Deep Learning. It is still when dealing with visualizations (UI).

In [6]:
# Parameters

# imgUrl = 'https://i.imgur.com/hfvNpmg.png'
imgUrl = 'https://i.postimg.cc/fyNcYwTm/hfvNpmg.png'

In [7]:
# Load Image

mI = ski.util.img_as_float(ski.io.imread(imgUrl))


In [ ]:
# Display Image

plt.imshow(mI);

### Image Masking

This section builds a UI to mask the image using the following _color spaces_:

1. `RGB` (Base).
2. `HSV` (SciKit Image).
3. `LAB` (SciKit Image).
4. `LCH` (SciKit Image).
5. `YCgCr` (Self Implementation).

The UI will have 3 range sliders per channel and a dropdown box to choose the _color space_.

* <font color='brown'>(**#**)</font> When color accuracy is crucial there are many factors to take under consideration: Gamma Curve, Device Calibration, Lightning Conditions, etc...

#### The YCgCr Color Space

The `YCgCr` color space is a linear transformation of the RGB color space using the following Matrix:

$$ \boldsymbol{C}_{YCgCr} = \begin{bmatrix} \phantom{+}0.25 & \phantom{+}0.50 & \phantom{+}0.25 \\ \phantom{+}0.50 & \phantom{+}0.00 & -0.50 \\ -0.25 & \phantom{+}0.50 & -0.25 \end{bmatrix} $$

* <font color='red'>(**?**)</font> Looking at the matrix, what are the advantages of it vs. other linear transformations?
* <font color='red'>(**?**)</font> What are the ranges of `Y`, `Cg` and `Cr?`
* <font color='red'>(**?**)</font> How would you implement this transformation for data type: `uin8`, `uint16`, `uint32`, etc...?



In [9]:
#===========================Fill This===========================#
# 1. Implement a function which converts from RGB to YCgCr.
# 2. The input is RGB in the range [0, 1].
# !! Numba can be used.

@njit
def RgbToYCgCr( mI: np.ndarray, mO: np.ndarray ):
    """
    Converts RGB image to YCgCr.
    Input:
      - mI           : A NumPy RGB image array in the range [0, 1].
      - mO           : A NumPy YCgCr image array.
    Remarks:
      - The input and output are floating point array.
    """
    
    for ii in range(mI.shape[0]):
        for jj in range(mI.shape[1]):
            mO[ii, jj, 0] =  0.25 * mI[ii, jj, 0] + 0.50 * mI[ii, jj, 1] + 0.25 * mI[ii, jj, 2]
            mO[ii, jj, 1] =  0.50 * mI[ii, jj, 0]                        - 0.50 * mI[ii, jj, 2]
            mO[ii, jj, 2] = -0.25 * mI[ii, jj, 0] + 0.50 * mI[ii, jj, 1] - 0.25 * mI[ii, jj, 2]

#===============================================================#

In [10]:
#===========================Fill This===========================#
# 1. Implement a Mask function which a product of the mask of each channel.
# 2. The function should support the listed color spaces.
# 3. The generated mask is smooth.
# !! To generate a smooth mask, use roll off at the edges of the range.
# !! Numba can be used.

def GenerateMask( mI: np.ndarray, colorSpace: int, lClip: List[Tuple[float, float]], mM: np.ndarray, rollOffWidth: float = 0.1 ):
    """
    Generate a luminosity mask for the input luminosity image.
    Input:
      - mI           : A NumPy image array (RGB).
      - colorSpace   : An integer: 0 -> RGB, 1 -> HSV, 2 -> LAB, 3 -> LCH, 4 -> YCgCr.
      - lClip        : A list of 3 tuples which define the mask range per channel.
      - mM           : A NumPy 2D array as a mask.
    Remarks:
      - The output range should be [0, 1].
      - The clips are always in the range [0, 1].  
        Data is scaled to match.
      - The selection roll off is smooth.
    """
    
    # Color Transform
    if colorSpace == 0: #<! RGB
        mC = mI.copy()
    elif colorSpace == 1: #<! HSV
        mC = ski.color.rgb2hsv(mI)
    elif colorSpace == 2: #<! LAB
        mC = ski.color.rgb2lab(mI)
    elif colorSpace == 3: #<! LCH
        mC = ski.color.lab2lch(ski.color.rgb2lab(mI))
    elif colorSpace == 4: #<! YCgCr
        mC = np.empty_like(mI)
        RgbToYCgCr(mI, mC)

    # Scaling
    # Scales each channel into the [0, 1] range.
    mC[:] = (mC - np.min(mC, axis = (0, 1))) / (np.max(mC, axis = (0, 1)) - np.min(mC, axis = (0, 1)))

    # Mask
    for ii in range(mC.shape[2]):
        GenSmoothRect(mC[:, :, ii], lClip[ii][0], lClip[ii][1], mC[:, :, ii], rollOffWidth = rollOffWidth)
    
    mM[:] = np.prod(mC, axis = 2)


@njit
def GenSmoothRect( vX: np.ndarray, lowVal: float, highVal: float, vY: np.ndarray, rollOffWidth: float = 0.1 ):
    # Smooth rectangle with roll off width
    
    lowClip  = max(lowVal - rollOffWidth, 0)
    highClip = min(highVal + rollOffWidth, 1)

    for ii in range(vX.size):
        valX = vX.flat[ii]
        if valX < lowClip:
            vY.flat[ii] = 0.0
        elif valX < lowVal:
            # Smoothstep [lowClip, lowVal]
            valXN = (lowVal - valX) / (lowVal - lowClip)
            vY.flat[ii] = 1 - (valXN * valXN * (3 - (2 * valXN)))
        elif valX > highClip:
            vY.flat[ii] = 0.0
        elif valX > highVal:
            # Smoothstep [highVal, highClip]
            valXN = (valX - highVal) / (highClip - highVal)
            vY.flat[ii] = 1 - (valXN * valXN * (3 - (2 * valXN)))
        else:
            vY.flat[ii] = 1.0


#===============================================================#

In [11]:
# Visualization Mask Image

def VisImgMask( mI: np.ndarray, colorSpaceStr: str, lClip: List[Tuple[float, float]], mM: np.ndarray, mO: np.ndarray ):

    dColorSpaceStr = {'RGB': 0, 'HSV': 1, 'LAB': 2, 'LCH': 3, 'YCgCr': 4}
    
    # Generate the Mask
    mM = np.empty(shape = mI.shape[:2])
    GenerateMask(mI, dColorSpaceStr[colorSpaceStr], lClip, mM)
    
    # Apply Mask
    mO[:] = mI * mM[:, :, None]
    
    hF, vHa = plt.subplots(nrows = 1, ncols = 3, figsize = (16, 6))
    
    vHa[0].imshow(mI)
    vHa[0].set(xticks = [], yticks = [])
    vHa[1].imshow(mM, cmap = 'gray', vmin = 0, vmax = 1)
    vHa[1].set(xticks = [], yticks = [])
    vHa[2].imshow(mO)
    vHa[2].set(xticks = [], yticks = [])

In [ ]:
# Interactive Widget 

mM = np.zeros_like(mI.shape[:2])
mO = np.zeros_like(mI)

hVisImgMask = lambda colorSpaceStr, tuChnl1Clip, tuChnl2Clip, tuChnl3Clip: VisImgMask(mI, colorSpaceStr, [tuChnl1Clip, tuChnl2Clip, tuChnl3Clip], mM, mO)

chnl1RangeSlider = FloatRangeSlider(value = (0.25, 0.75), min = 0, max = 1, step = 0.01, 
                                  description = 'Channel 1 Range:', continuous_update = False, 
                                  orientation = 'horizontal', readout = True, readout_format = '0.2f', 
                                  layout = Layout(width = '20%'), style = {'description_width': 'initial'})
chnl2RangeSlider = FloatRangeSlider(value = (0.25, 0.75), min = 0, max = 1, step = 0.01, 
                                  description = 'Channel 2 Range:', continuous_update = False, 
                                  orientation = 'horizontal', readout = True, readout_format = '0.2f', 
                                  layout = Layout(width = '20%'), style = {'description_width': 'initial'})
chnl3RangeSlider = FloatRangeSlider(value = (0.25, 0.75), min = 0, max = 1, step = 0.01, 
                                  description = 'Channel 3 Range:', continuous_update = False, 
                                  orientation = 'horizontal', readout = True, readout_format = '0.2f', 
                                  layout = Layout(width = '20%'), style = {'description_width': 'initial'})
colorSpaceStrDropdown = Dropdown(options = ['RGB', 'HSV', 'LAB', 'LCH', 'YCgCr'], value = 'RGB', description = 'Luminosity Mask Type:', 
                                 style = {'description_width': 'initial'})
interact(hVisImgMask, colorSpaceStr = colorSpaceStrDropdown, 
         tuChnl1Clip = chnl1RangeSlider, tuChnl2Clip = chnl2RangeSlider, tuChnl3Clip = chnl3RangeSlider)

plt.show()